In [2]:
import requests
import pandas as pd
import sys
from pandas.io.json import json_normalize
from datetime import datetime
import gc
from google.oauth2 import credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.cloud import bigquery

In [9]:
# Set the scopes for the desired permissions
SCOPES = ['https://www.googleapis.com/auth/bigquery']

# Create the flow for authorization
flow = InstalledAppFlow.from_client_secrets_file(
    '...', scopes=SCOPES
)
credentials = flow.run_local_server(port=0)

project_id = '...'

# Use the obtained credentials for authentication
client = bigquery.Client(credentials=credentials, project=project_id) #

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=853561264369-cg46opisk56diiv2ee4nblkb6jmtland.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A61007%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&state=cfrpop9xpQGIKQ8u8UfGzXEH51Coud&access_type=offline


In [21]:
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "supple-century-387419-a4bd08df53b7.json"

In [10]:
# Construct a reference to the "hacker_news" dataset
dataset_ref = client.dataset("openaq", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

In [11]:
# Construct a reference to the "global_air_quality" table
table_ref = dataset_ref.table("global_air_quality")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the "global_air_quality" table
client.list_rows(table, max_results=5).to_dataframe()

,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours,location_geom
0,"Borówiec, ul. Drapałka",Borówiec,PL,bc,0.85217,2022-04-28 07:00:00+00:00,µg/m³,GIOS,1.0,52.276794,17.074114,POINT(52.276794 1)
1,"Kraków, ul. Bulwarowa",Kraków,PL,bc,0.91284,2022-04-27 23:00:00+00:00,µg/m³,GIOS,1.0,50.069308,20.053492,POINT(50.069308 1)
2,"Płock, ul. Reja",Płock,PL,bc,1.41000,2022-03-30 04:00:00+00:00,µg/m³,GIOS,1.0,52.550938,19.709791,POINT(52.550938 1)
3,"Elbląg, ul. Bażyńskiego",Elbląg,PL,bc,0.33607,2022-05-03 13:00:00+00:00,µg/m³,GIOS,1.0,54.167847,19.410942,POINT(54.167847 1)
4,"Piastów, ul. Pułaskiego",Piastów,PL,bc,0.51000,2022-05-11 05:00:00+00:00,µg/m³,GIOS,1.0,52.191728,20.837489,POINT(52.191728 1)


## SQL

In [12]:
# Query to select all the items from the "city" column where the "country" column is 'US'
query = """
        SELECT city
        FROM `bigquery-public-data.openaq.global_air_quality`
        WHERE country = 'US'
        """

In [59]:
client.list_rows(table, max_results=5).to_dataframe()

,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours,location_geom
0,"Borówiec, ul. Drapałka",Borówiec,PL,bc,0.85217,2022-04-28 07:00:00+00:00,µg/m³,GIOS,1.0,52.276794,17.074114,POINT(52.276794 1)
1,"Kraków, ul. Bulwarowa",Kraków,PL,bc,0.91284,2022-04-27 23:00:00+00:00,µg/m³,GIOS,1.0,50.069308,20.053492,POINT(50.069308 1)
2,"Płock, ul. Reja",Płock,PL,bc,1.41000,2022-03-30 04:00:00+00:00,µg/m³,GIOS,1.0,52.550938,19.709791,POINT(52.550938 1)
3,"Elbląg, ul. Bażyńskiego",Elbląg,PL,bc,0.33607,2022-05-03 13:00:00+00:00,µg/m³,GIOS,1.0,54.167847,19.410942,POINT(54.167847 1)
4,"Piastów, ul. Pułaskiego",Piastów,PL,bc,0.51000,2022-05-11 05:00:00+00:00,µg/m³,GIOS,1.0,52.191728,20.837489,POINT(52.191728 1)


### High air quality
Which pollution levels were reported to be exactly 0?

Solving through pandas

In [58]:
data=client.list_rows(table).to_dataframe()

In [60]:
data.columns

Index(['location', 'city', 'country', 'pollutant', 'value', 'timestamp',
       'unit', 'source_name', 'latitude', 'longitude',
       'averaged_over_in_hours', 'location_geom'],
      dtype='object')

In [62]:
data[data['value']==0]

,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours,location_geom
42,"Koszalin, ul. Armii Krajowej",Koszalin,PL,bc,0.0,2022-05-05 01:00:00+00:00,µg/m³,GIOS,1.0,54.193986,16.172544,POINT(54.193986 1)
95,"Żary, ul. Szymanowskiego 8",Żary,PL,bc,0.0,2022-05-02 12:00:00+00:00,µg/m³,GIOS,1.0,51.642656,15.127808,POINT(51.642656 1)
105,"Kraków, ul. Bulwarowa",Kraków,PL,bc,0.0,2022-04-08 04:00:00+00:00,µg/m³,GIOS,1.0,50.069308,20.053492,POINT(50.069308 1)
126,Gorzów Wlkp. ul. Kosynierów Gdyńskich,Gorzów Wielkopolski,PL,bc,0.0,2022-05-12 16:00:00+00:00,µg/m³,GIOS,1.0,52.738214,15.228667,POINT(52.738214 1)
151,"Zielonka, Bory Tucholskie",Zielonka,PL,bc,0.0,2022-04-16 04:00:00+00:00,µg/m³,GIOS,1.0,53.662136,17.933986,POINT(53.662136 1)
...,...,...,...,...,...,...,...,...,...,...,...,...
5594326,City Hall - Durban-NAQI,eThekwini Metro,ZA,pm25,0.0,2022-05-14 19:00:00+00:00,µg/m³,South Africa,1.0,-29.858283,31.027286,POINT(-29.858283 1)
5594336,Lebohang,Gert Sibande,ZA,pm25,0.0,2022-05-22 09:00:00+00:00,µg/m³,South Africa,1.0,-26.381111,28.918333,POINT(-26.381111 1)
5594359,Hendrina - SAWS,Nkangala,ZA,pm25,0.0,2022-05-16 18:00:00+00:00,µg/m³,South Africa,1.0,-26.151197,29.716484,POINT(-26.151197 1)
5594394,Ermelo-NAQI,Gert Sibande,ZA,pm25,0.0,2022-05-19 18:00:00+00:00,µg/m³,South Africa,1.0,-26.493348,29.968054,POINT(-26.493348 1)


In [14]:
# Set up the query
query_job = client.query(query)

In [15]:
# API request - run the query, and return a pandas DataFrame
us_cities = query_job.to_dataframe()

In [24]:
# What five cities have the most measurements?
us_cities.city.value_counts(ascending=False).head(5)

Phoenix-Mesa-Scottsdale                     39414
Los Angeles-Long Beach-Santa Ana            27479
Riverside-San Bernardino-Ontario            26887
New York-Northern New Jersey-Long Island    25417
San Francisco-Oakland-Fremont               22710
Name: city, dtype: int64

In [26]:
query1 = """
        SELECT city, country
        FROM `bigquery-public-data.openaq.global_air_quality`
        WHERE country = 'US'
        """

In [28]:
query2 = """
        SELECT *
        FROM `bigquery-public-data.openaq.global_air_quality`
        WHERE country = 'US'
        """

## Exploring Data

### Find countries that have reported pollution levels in units of "ppm

In [ ]:
# Query to get the score column from every row where the type column has value "job"
query4 = """
        SELECT country
        FROM `bigquery-public-data.openaq.global_air_quality`
        WHERE unit = 'ppm'
        """

# Create a QueryJobConfig object to estimate size of query without running it
dry_run_config = bigquery.QueryJobConfig(dry_run=True)

# API request - dry run query to estimate costs
dry_run_query_job = client.query(query4, job_config=dry_run_config)

print("This query will process {} bytes.".format(dry_run_query_job.total_bytes_processed))

In [34]:
query_job2 = client.query(query4)

In [36]:
country_ppm = query_job2.to_dataframe()

In [38]:
country_ppm.head()

,country
0,AR
1,AR
2,AR
3,AR
4,AR


In [39]:
country_ppm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1286109 entries, 0 to 1286108
Data columns (total 1 columns):
 #   Column   Non-Null Count    Dtype 
---  ------   --------------    ----- 
 0   country  1286109 non-null  object
dtypes: object(1)
memory usage: 9.8+ MB


In [40]:
country_ppm.value_counts()

country
US         816326
CA         168284
TH          92819
ZA          72156
AU          63740
MX          59073
BR           7506
AR           1745
GT            995
IL            708
TW            630
AE            520
NP            495
QA            366
CH            349
CO            302
PE             52
EC             21
UZ              5
BM              3
MM              2
IN              2
RW              2
GB              2
BH              2
CW              2
CL              1
RO              1
dtype: int64

### High air quality
Which pollution levels were reported to be exactly 0?

Solving in SQL and confirming pandas results


In [41]:

query5 = """
                       SELECT *
                       FROM `bigquery-public-data.openaq.global_air_quality`
                       WHERE value = 0
                       """

# Create a QueryJobConfig object to estimate size of query without running it
dry_run_config = bigquery.QueryJobConfig(dry_run=True)

# API request - dry run query to estimate costs
dry_run_query_job = client.query(query, job_config=dry_run_config)

print("This query will process {} bytes.".format(dry_run_query_job.total_bytes_processed))

This query will process 67585546 bytes.


In [53]:
query_job3 = client.query(query5)
zero_polution = query_job3.to_dataframe()

In [54]:
zero_polution

,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours,location_geom
0,"Zielonka, Bory Tucholskie",Zielonka,PL,bc,0.0,2022-04-29 14:00:00+00:00,µg/m³,GIOS,1.0,53.662136,17.933986,POINT(53.662136 1)
1,"Toruń, ul. Przy Kaszowniku",Toruń,PL,bc,0.0,2022-04-19 04:00:00+00:00,µg/m³,GIOS,1.0,53.017628,18.612808,POINT(53.017628 1)
2,"Kielce, ul. Targowa",Kielce,PL,bc,0.0,2022-05-07 17:00:00+00:00,µg/m³,GIOS,1.0,50.878998,20.633692,POINT(50.878998 1)
3,"Zielonka, Bory Tucholskie",Zielonka,PL,bc,0.0,2022-05-19 14:00:00+00:00,µg/m³,GIOS,1.0,53.662136,17.933986,POINT(53.662136 1)
4,"Koszalin, ul. Armii Krajowej",Koszalin,PL,bc,0.0,2022-05-12 20:00:00+00:00,µg/m³,GIOS,1.0,54.193986,16.172544,POINT(54.193986 1)
...,...,...,...,...,...,...,...,...,...,...,...,...
192702,Middelburg SAWS-NAQI,Nkangala,ZA,pm25,0.0,2022-05-22 20:00:00+00:00,µg/m³,South Africa,1.0,-25.796056,29.462823,POINT(-25.796056 1)
192703,Leitrim,Fezile Dabi,ZA,pm25,0.0,2022-04-02 03:00:00+00:00,µg/m³,South Africa,1.0,-26.851461,27.875845,POINT(-26.851461 1)
192704,Silobela,Gert Sibande,ZA,pm25,0.0,2021-08-31 21:00:00+00:00,µg/m³,South Africa,1.0,-26.086232,30.110653,POINT(-26.086232 1)
192705,eMbalenhle North,Gert Sibande,ZA,pm25,0.0,2022-05-14 18:00:00+00:00,µg/m³,South Africa,1.0,-26.536389,29.072500,POINT(-26.536389 1)
